<a href="https://colab.research.google.com/github/Prem-2407/Ted-Talks/blob/main/Almabetter_project_Ted_Talks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Title:  Ted Talks Views Prediction

---



TED is decoted to spereading powerful ideas on just about any topic . These datasets contain over 4,000 talks including transcripts in many languages.

Founded in 1984 by Richard Salman as a nonprofit organization that aimed at bringing experts fromm the fileds of Technology ,Entertainmnet and Design together ..

TED conferences have gone on to becomme theMecca of ideas from virtually all walks of life .As of 2015 ,TED and its sister TEDx chapters have published more than 2000 talks for free consumption by the masses  and its speaker list boasts of the likes of AI Gore, Jimmy Wales,ShahRukh Khan,Bll Gates.


Our main objective id to build a predictive model to predict the number of views for the respective videos from the TEDx website.As we have a continous varible to predict ,we use regression models to predict the views of the TED talks .

#**Steps to follow** 

**1) Data Loading**

**2)Data Cleaning**

**3)EDA**

**4) Feature engineering**

**5) Model building**

In [ ]:
  # Importing the libraries
import numpy as np
import pandas as pd
from numpy import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#**Mounting The Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**Loading the Data set**

In [ ]:
tedtalk_df=pd.read_csv('//content/drive/MyDrive/Alma Better Ted Talks/data_ted_talks.csv')

In [ ]:
tedtalk_df.head(3)

In [ ]:
tedtalk_df.columns

In [ ]:
tedtalk_df.info()

**Shape of the Dataset**

In [ ]:
tedtalk_df.shape

By the shape we can get that there are **4005 Rows** Along with **19 variables** 

#**Data Cleaning** 

**Missing values**

In [ ]:
def missing_values_table(df):
        mis_val =df.isna().sum()
        mis_val_percent = 100 *df.isna().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table.sort_values('% of Total Values', ascending=False).round(1)
        print (" selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]),"\n\n")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_values_table(tedtalk_df)

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(tedtalk_df.isnull(),cbar=False,yticklabels=False,cmap='OrRd');

We can see that we have lotof missing values in several columns ,Therefore we will be treated after the EDA

Here many columns have the data in the form of a string and dictionaries .To acess the data and to build model,we sould have cleaned data which can be easily acessed .So we are converting all colummns to its proper datatype

In [ ]:
tedtalk_df['occupations'].fillna(str({0:['other']}),inplace=True)
#filling NA with other

tedtalk_df['occupations'] = tedtalk_df.apply(lambda row: eval(row['occupations']), axis=1)
#to convert from str to dict

tedtalk_df['occupations'] = tedtalk_df['occupations'].apply(lambda x: x.get(0))
#to extract the dict values

tedtalk_df['about_speakers'].fillna(str({0:'Not specified'}),inplace=True)
#filling the NA's with 'Not specified'

tedtalk_df['about_speakers'] = tedtalk_df.apply(lambda row: eval(row['about_speakers']), axis=1)
#to convert from str to dict

tedtalk_df['about_speakers'] = tedtalk_df['about_speakers'].apply(lambda x: x.get(0))
#to extract the dict values

tedtalk_df['recorded_date'].fillna(tedtalk_df['recorded_date'].mode()[0],inplace=True)
#since dates are kind of discrete vaariables, we replace NA with mode.

tedtalk_df['available_lang'] = tedtalk_df['available_lang'].apply(lambda row: eval(row))
tedtalk_df['topics'] = tedtalk_df['topics'].apply(lambda row: eval(row))
tedtalk_df['related_talks'] = tedtalk_df['related_talks'].apply(lambda row: eval(row))
#converting all the remaining columns which were in strings to their respective datatypes.

Here ,some of the mmissing values are treated which missinng values % was very less.its done becaues it will give better visualizations in our EDA .

In [ ]:
tedtalk_df.drop(['talk_id','all_speakers','url'],axis=1,inplace=True)

These columns are deleted as its not carrying much significance

In [ ]:
tedtalk_df.info()

Date ,Month and year columns are converted to its proper Datetimme datatype.

In [ ]:
import datetime
tedtalk_df['published_date']=pd.to_datetime(tedtalk_df['published_date'], format='%Y-%m-%d')
tedtalk_df['recorded_date']=pd.to_datetime(tedtalk_df['recorded_date'], format='%Y-%m-%d')
#converting the published and recorded date as datetime datatype

In [ ]:
tedtalk_df['release_day'] = tedtalk_df['published_date'].apply(lambda x: x.weekday())
tedtalk_df['release_month']=tedtalk_df['published_date'].apply(lambda x: x.month)
tedtalk_df['release_year'] = tedtalk_df['published_date'].apply(lambda x: x.year)
#finding out the respected day,month and year

In [ ]:
week_day={0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
tedtalk_df['release_day']=tedtalk_df['release_day'].map(week_day)
#tedtalk_day was interms of numbers which were holding the record of monday till sunday...so converting that to exact days here

month_dict={1:'Jan',2:'Feb',3:'March',4:'April',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
tedtalk_df['release_month']=tedtalk_df['release_month'].map(month_dict)
#again converting the month which was in numbers to exact month names

In [ ]:
tedtalk_df.head(2)

In [ ]:
ted_talk_df = tedtalk_df.copy()

In [ ]:
tedtalk_df.head(2)

Now that a basic level cleaning of dataet is done we will start with the processs of EDA 

# EDA

# Speaker_1_column

**Speakers with top 20 total views wrt their numbero f talks**

In [ ]:
tedtalk_df['speaker_1'].nunique()

In [ ]:
speaker_with_most_talks = pd.DataFrame(tedtalk_df['speaker_1'].value_counts()).reset_index()
speaker_with_most_talks.rename(columns={'index':'speaker_1', 'speaker_1':'num_of_talks'}, inplace=True)


speaker_total_views_df=tedtalk_df.groupby('speaker_1').agg({'views' : 'sum'}).reset_index()
#speaker with respected total views
speaker_tv_df = speaker_total_views_df.head(20).sort_values('views',ascending=False)

speaker_report = speaker_with_most_talks.merge(speaker_total_views_df,on='speaker_1').reset_index()

In [ ]:
#taking only numeric columns to do the minmaxscaling
speaker_numeric = speaker_report[['num_of_talks','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(speaker_numeric)
#print(scaled)
speaker_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
speaker_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))
speaker_report = speaker_report.head(20)

In [ ]:
# graph for speakers which has maximum talks with respect to views

# # plotting the graph

speaker_report_1 = speaker_report.sort_values('num_of_talks_scaler')
speaker_report_2 = speaker_report.sort_values('views')
plt.figure(figsize=(20,6))

plt.subplot(1, 2, 1)
#plt.figure(figsize=(20,6))
plt.xticks(rotation=90)
sns.barplot(x='speaker_1',y='num_of_talks_scaler',data = speaker_report_1)
sns.lineplot(x='speaker_1',y='views_scaler',data=speaker_report_1,marker='o')
plt.title('Speaker with top 10 number of talks and total views')

plt.subplot(1, 2, 2)
plt.xticks(rotation=90)
sns.barplot(x='speaker_1',y='num_of_talks_scaler',data = speaker_report_2)
sns.lineplot(x='speaker_1',y='views_scaler',data=speaker_report_2,marker='o')
plt.title('Speaker with top 10 total views and number_of_talks')
#speaker_report

We have the visualization for the same data from two different aspect here.

1) First is the graph which contains speakerswho has given most numbers of talks and the line graph represents their views.We can observe that Bill Gates was the influneced  person. So even if his number of taks are less, he has received more number of views .So the speaker influences the number of views of the Talks.We can see somme hike in the graph for the influneced speakers.
 
 2) second is the graph which is stored according to the number odf vviews talks received .We van obser e that the number of talks is not directly contributing to the number of views talks can receive.

#**Speaker who received more number of views for one video**

In [ ]:
most_popular_video_df = tedtalk_df.nlargest(10,['views'])
most_popular_video_df[['speaker_1','views','title']]

In [ ]:
plt.figure(figsize=(15,7))
plt.title("most views received for the single video by speaker")
sns.barplot(x='speaker_1',y='views',data=most_popular_video_df)

This graph represents the maximum views a single video recived .Sir Ken Robinson is the ione whi recived maxximmum number for a single video 

#**Occupations column**

**Top occupations of the speaker**

In [ ]:

tedtalk_df['speaker_occupation']=pd.Series()
for i in range(len(tedtalk_df)):
  tedtalk_df.loc[i,'speaker_occupation'] = tedtalk_df['occupations'][i][0]

In [ ]:
tedtalk_df['speaker_occupation']

In [ ]:
tedtalk_df.drop('occupations',axis=1,inplace=True)

In [ ]:
tedtalk_df['speaker_occupation'].nunique()

By this we can observe that there are 1552 different occupations for the respective speakers in TED talk videos 

In [ ]:
speaker_occupations_df = pd.DataFrame(tedtalk_df['speaker_occupation'].value_counts()).reset_index()
speaker_occupations_df.rename(columns={'index':'Occupations', 'speaker_occupation':'Number_of_speakers'}, inplace=True)

occupations_with_most_views = tedtalk_df.groupby(['speaker_occupation'],dropna=True)['views'].mean().reset_index()
occupations_with_most_views.rename(columns={'speaker_occupation':'Occupations', 'views':'views'}, inplace=True)

occupation_report = speaker_occupations_df.merge(occupations_with_most_views,on='Occupations')


In [ ]:
#taking only numeric columns to do the minmaxscaling
occupation_numeric = occupation_report[['Number_of_speakers','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(occupation_numeric)
#print(scaled)
occupation_report ['Number_of_speakers_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
occupation_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))
occupation_report = occupation_report.loc[1:21,:]

In [ ]:
# graph for occupation which has maximum talks with respect to views

# # plotting the graph

occupation_report_1 = occupation_report.sort_values('Number_of_speakers')
occupation_report_2 = occupation_report.sort_values('views')

plt.figure(figsize=(20,6))

plt.subplot(1, 2, 1)
#plt.figure(figsize=(20,6))
plt.xticks(rotation=90)
sns.barplot(x='Occupations',y='Number_of_speakers_scaler',data = occupation_report_1)
sns.lineplot(x='Occupations',y='views_scaler',data = occupation_report_1,marker='o')
plt.title('Occupations with number of speakers and respected average views')



plt.subplot(1, 2, 2)
#plt.figure(figsize=(20,6))
plt.xticks(rotation=90)
sns.barplot(x='Occupations',y='Number_of_speakers_scaler',data = occupation_report_2)
sns.lineplot(x='Occupations',y='views_scaler',data = occupation_report_2,marker='o')
plt.title('Occupations with average views and respected number of speakers')



Again we have 2 representation of the same graphs, once sorted according to the number of talks then according to the number of views.

(1) In first graph, we can see that, again number of talks for the respective occupations are not directly contributing to the number of views. Instead the type of occupations are bringinfg a hike in number of views. For example, psychology (which everybody loves to listen about) and Activist (Who will make everybody listen to them by theri words). The occupations (might be their skills to talk to) does matter in number of views.

(2) Second graph contains the grapph which is sorted according to the number of views. We can See that the top 3 views were received by Psychologist, Activist and Author who all are very good in making people love what they speak.

#**Occupations which received maximum number of views for a single video**

In [ ]:

most_popular_video_df = tedtalk_df.nlargest(10,['views'])
most_popular_video_df[['speaker_occupation','views','title']]

In [ ]:
plt.figure(figsize=(15,7))
plt.title("most viewd video_Occupation")
plt.xticks(rotation=90)
sns.barplot(x='speaker_occupation',y='views',data=most_popular_video_df)

we can observe that the maximum number of views for a single video was received by the occuppations Author followed by Psychologist .

#**Published date and released date**

**Findiing out on which day most of the vidoes were released**

In [ ]:
#creating a df which holds the number of talks on each day
talk_rel_day_df = pd.DataFrame(tedtalk_df['release_day'].value_counts()).reset_index()
talk_rel_day_df.rename(columns={'index':'release_day', 'release_day':'num_of_talks'}, inplace=True)

#creating another df which holds the record of views on each day
popular_day = tedtalk_df[['release_day','views']].groupby('release_day').agg({'views' : 'mean'}).sort_values('views',ascending=False).reset_index()

#merging both df
talk_day_report = talk_rel_day_df.merge(popular_day,on='release_day')

talk_day_report

In [ ]:
#Sorting the days by using another dataframe with the day values

day_value = pd.DataFrame([['Sunday',0],['Monday',1],['Tuesday',2],['Wednesday',3],['Thursday',4],['Friday',5],['Saturday',6]])
day_value.rename(columns={0:'release_day',1:'Day_value'},inplace=True)
day_value
talk_day_report = talk_day_report.merge(day_value,on='release_day')

talk_day_report

In [ ]:
talk_day_report = talk_day_report.sort_values('Day_value',ascending=True)
talk_day_report = talk_day_report.drop('Day_value',axis=True)
talk_day_report

In [ ]:
#taking only numeric columns to do the minmaxscaling
talk_day_numeric = talk_day_report[['num_of_talks','views']]
print(talk_day_numeric)
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(talk_day_numeric)
#print(scaled)
talk_day_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
talk_day_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))

In [ ]:
talk_day_report

In [ ]:
plt.figure(figsize=(16,8))
for i in talk_day_report.iloc[:,3:]:
  sns.lineplot(data=talk_day_report,x='release_day',y=i,marker='o')

plt.legend(labels=talk_day_numeric)


We can observe in this graph about the comparision of number of talks and as well as the number of views a particular day received. We can see that on saturday the number of talks and as well as the number of views were low. And on Sunday, Wednesday and Friday the release were high. But the number of views were high only on Sunday.

It may seem that only on weekends people were interested to give a time for TED talk videos.

#**Month**

In [ ]:
! pip install sorted-months-weekdays
! pip install sort-dataframeby-monthorweek

from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *

In [ ]:
# creating data frame for num of talks in a particular month 
talk_rel_month_df = pd.DataFrame(tedtalk_df['release_month'].value_counts()).reset_index()
talk_rel_month_df.rename(columns={'index':'release_month', 'release_month':'num_of_talks'}, inplace=True)

#Printing the most popular release month according to average Views
popular_month = tedtalk_df[['release_month','views']].groupby('release_month').agg({'views' : 'mean'}).sort_values('release_month').reset_index()

#merging two dataframes which has number of talks and views received in the same df
talk_month_report = talk_rel_month_df.merge(popular_month,on='release_month')

#sorting according to month in here
talk_month_report = Sort_Dataframeby_Month(df=talk_month_report ,monthcolumnname='release_month')



In [ ]:
#taking only numeric columns to do the minmaxscaling
talk_month_numeric = talk_month_report[['num_of_talks','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(talk_month_numeric)
#print(scaled)
talk_month_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
talk_month_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))

In [ ]:

talk_month_report

In [ ]:
plt.figure(figsize=(16,8))
for i in talk_month_report.iloc[:,3:]:
  sns.lineplot(data=talk_month_report,x='release_month',y=i,marker='o')

plt.legend(talk_month_numeric)


Here we can observe that it has the comparision wise line graph for number of talks and number of views for the respective month.

Most number of videos wwere released on the month of April where the lease was released on the month of August. But the number of views were maximum for the videos which were released on the minth of March and it was lease for the videos of Novemb

#**YEAR**

In [ ]:
talk_rel_year_df = pd.DataFrame(tedtalk_df['release_year'].value_counts()).reset_index()
talk_rel_year_df.rename(columns={'index':'release_year', 'release_year':'num_of_talks'}, inplace=True)

popular_year = tedtalk_df[['release_year','views']].groupby('release_year').agg({'views' : 'mean'}).sort_values('release_year',ascending=False).reset_index()

talk_year_report = talk_rel_year_df.merge(popular_year,on='release_year').sort_values('release_year')


In [ ]:
#taking only numeric columns to do the minmaxscaling
talk_year_numeric = talk_year_report[['num_of_talks','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(talk_year_numeric)
#print(scaled)
talk_year_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
talk_year_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))

In [ ]:
talk_year_report

In [ ]:
plt.figure(figsize=(16,8))
for i in talk_year_report.iloc[:,3:]:
  sns.lineplot(data=talk_year_report,x='release_year',y=i,marker='o')
plt.legend(labels=talk_year_numeric)

We can observe the line graph for almost 20 years of number of talks and number of views. We can conclude that Number of talks were morein old days but eventually it was reduced hit the minimum in 2019. It hits the hike in 2007 though. But the number of views were gradually increasing from year to year with a little bit of ups and downs. It was high during the 2019 when the number of talks were minimum.

**To view the number of views recieved per day on monthly basis**

In [ ]:
from datetime import date
graph_df = tedtalk_df.copy()
g = sns.FacetGrid(graph_df, col='release_day', height=8, aspect=.5)
td = pd.to_datetime(date.today(), format='%Y-%m-%d')
graph_df['video_age'] = (td - tedtalk_df['published_date']).apply(lambda x: x.days)
graph_df['views_per_day'] = graph_df['views'] / ( graph_df['video_age'] + 1 )
g.map(sns.barplot, 'release_month', 'views_per_day')


**Comment and duration columns**

In [ ]:
sns.jointplot(data=tedtalk_df,x='comments', y='views')

In [ ]:
sns.jointplot(data=tedtalk_df,x='duration', y='views')

In [ ]:
plt.figure(figsize=(20,6))

plt.subplot(1,3,1)
sns.distplot(tedtalk_df['views'])
plt.subplot(1,3,2)
sns.distplot(tedtalk_df['comments'])
plt.subplot(1,3,3)
sns.distplot(tedtalk_df['duration'])


we can observe the data distribution of the numerical columns views ,comments and duration

#**Events with respect to talk and views**

In [ ]:
#count of events

event_count_df = pd.DataFrame(tedtalk_df['event'].value_counts()).reset_index()
event_count_df.rename(columns={'index':'event', 'event':'num_of_talks'}, inplace=True)

#event with max views
popular_event=tedtalk_df[['event','views']].groupby('event').agg({'views' : 'mean'}).sort_values('views',ascending=False).reset_index()

event_report = event_count_df.merge(popular_event,on='event')



In [ ]:
#taking only numeric columns to do the minmaxscaling
event_numeric = event_report[['num_of_talks','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(event_numeric)
#print(scaled)
event_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
event_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))

In [ ]:
# graph for event which has maximum talks with respect to views
event_talk_views_report_1 = event_report.sort_values('num_of_talks',ascending=False).head(20)
event_talk_views_report_2 = event_report.sort_values('views',ascending=False).head(20)
# # plotting the graph

plt.figure(figsize=(20,6))

plt.subplot(1,2,1)
plt.xticks(rotation=90)
sns.barplot(x='event',y='num_of_talks_scaler',data = event_talk_views_report_1)
sns.lineplot(x='event',y='views_scaler',data=event_talk_views_report_1,marker='o')
plt.title('Events with top 10 number of talks followed by their views')

plt.subplot(1,2,2)
plt.xticks(rotation=90)
sns.barplot(x='event',y='num_of_talks_scaler',data = event_talk_views_report_2)
sns.lineplot(x='event',y='views_scaler',data=event_talk_views_report_2,marker='o')
plt.title('Events with top 10 views followed by their number of talks')


This graph represents the number of views for the different views along with the number of talks. We can observe that the count of number of talks of event did not really matter for views. For top 10 events in the first graph, the number of views is almost the same. And in the second graph, for the top 10 nummber of views, event's number of talks were very low

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(event_report['event'],event_report['views'])

This explains about the diffrent events with respect to the nummer of views .And we can see some hikes in number of views for very few events in here .

#**Native lang column**

In [ ]:
tedtalk_df['native_lang'].nunique()

In [ ]:
native_lang_df = pd.DataFrame(tedtalk_df['native_lang'].value_counts()).reset_index()

native_lang_df.rename(columns={'index':'native_lang', 'native_lang':'num_of_talks'}, inplace=True)

#event with max views
popular_lang=tedtalk_df[['native_lang','views']].groupby('native_lang').agg({'views' : 'mean'}).sort_values('views',ascending=False).reset_index()

lang_report = native_lang_df.merge(popular_lang,on='native_lang')

In [ ]:
#taking only numeric columns to do the minmaxscaling
lang_numeric = event_report[['num_of_talks','views']]
#using standardization as both numeric columns are in different scale
#after words it will be easy for us to do the graphs
scaler = MinMaxScaler()
scaled = scaler.fit_transform(lang_numeric)
#print(scaled)
lang_report ['num_of_talks_scaler'] = pd.Series(scaled[i][0] for i in range(len(scaled)))
lang_report ['views_scaler'] = pd.Series(scaled[i][1] for i in range(len(scaled)))

In [ ]:
# graph for langauges which has maximum talks with respect to views
#lang_report.sort_values('views',ascending=False,inplace=True)
# # plotting the graph
lang_report_1 = lang_report.sort_values('num_of_talks',ascending=False)
lang_report_2 = lang_report.sort_values('views',ascending=False)

plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
sns.barplot(x='native_lang',y='num_of_talks_scaler',data = lang_report_1)
sns.lineplot(x='native_lang',y='views_scaler',data=lang_report_1,marker='o')
plt.title('Native languages with high number of talks followed by its views')

plt.subplot(1,2,2)
sns.barplot(x='native_lang',y='num_of_talks_scaler',data = lang_report_2)
sns.lineplot(x='native_lang',y='views_scaler',data=lang_report_2,marker='o')
plt.title('Native languages with top views followed by thier num_of_talks')

lang_report


We can see the different native languages avaliable, number of talks in those languages and number of views it received. In the first graph we can see the charts which are sorted according to the number of talks. Obviously english being the common langauge, has most numer of talks. But it did not ensure abou tthe number of views. The second graph contains the charts which are sorted according to number of views. And we can see that Portuguese has received maximum average views for its talks.

# Available lang column

**subtitles count for all videos**

**bold text**

In [ ]:
lang_dict = {}
for i in tedtalk_df.to_dict('records'):
   tmp = i['available_lang']
   for j in tmp:
    if j in lang_dict:
      lang_dict[j] += 1
    else:
      lang_dict[j] = 1 
lang_df = pd.DataFrame(lang_dict.values(),index=lang_dict.keys(),columns=['count']).head(20).reset_index().sort_values('count',ascending=False)



In [ ]:
lang_df

In [ ]:
plt.figure(figsize=(18,8))
plt.title('% of Videos dubbed in different languages')
ax = sns.barplot(x='index',y='count',data=lang_df)
plt.xlabel("Languages")
plt.ylabel('Counts')


This graph gives the idea about how many languages were availble as subtitles for the videos ,and again English commes in top as its the common language all over the world

**Topic column**

In [ ]:
tedtalk_df['topics']

In [ ]:
topics_df = pd.DataFrame(tedtalk_df['topics'].explode().value_counts().reset_index()).iloc[:,0:]
topics_df.rename(columns={'index':'Topics','topics':'Number_of_talks'},inplace=True)



In [ ]:
plt.figure(figsize=(18,8))
plt.xticks(rotation=90)
sns.barplot(x='Topics',y='Number_of_talks',data=topics_df.head(20))

we can see thet the most TED talks were about science and technology .


**Wordcloud for topics**

In [ ]:
from wordcloud import WordCloud
text = " ".join(topics_df['Topics'])
word_cloud = WordCloud(max_words=100,collocations = False, background_color = 'white').generate(text)

In [ ]:
plt.figure(figsize=(16,7))
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

**Description column**

In [ ]:
text = " ".join(tedtalk_df['description'])
word_cloud = WordCloud(max_words=100,collocations = False, background_color = 'black').generate(text)
plt.figure(figsize=(16,7))
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

**Title column**

In [ ]:
text = " ".join(tedtalk_df['title'])
word_cloud = WordCloud(max_words=100,collocations = False, background_color = 'red').generate(text)
plt.figure(figsize=(16,7))
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

**Heatmap**

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(round(tedtalk_df.corr(),2),annot=True,cmap='Greens');

Trying to get a correlation matrix with visualization for the numeric columns.We can observe no 2 column share correlation more than 60%

#**Missing values in comments**

In [ ]:
td_talk_df_2 = tedtalk_df.copy()

In [ ]:
sns.distplot(tedtalk_df.comments.dropna())

In [ ]:
numeric_df = tedtalk_df[['views','comments','duration']]

As more than 3% of the data were missing in comments ,we tried using the KNN imouter to predict the missing values ion here .

In [ ]:
#KNN to find the missing values

from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

# Doing dummy for categorical variables
numeric_df = pd.get_dummies(numeric_df)

# Defining scaler and imputer objects
scaler = StandardScaler()
imputer = KNNImputer()

# Imputing missing values with KNN if any
numeric_df['comments'] = imputer.fit_transform((numeric_df['comments'].values.reshape(-1,1)))

In [ ]:
sns.distplot(numeric_df['comments'])

In [ ]:
tedtalk_df['comments']=numeric_df['comments']
tedtalk_df.isna().sum()

In [ ]:
ted_talk_3_df = tedtalk_df.copy()


# **Handling Outliers**

In [ ]:
tedtalk_df.describe().T
#we can see that we have only numerical columns in here.
#talk_id is the unique number which should not be considered
#views is the dependent variable which should not be considered
# so we consider only comments and duration.

In [ ]:
continous = ['comments','duration']

In [ ]:
for var in continous:
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    fig = sns.boxplot(y=tedtalk_df[var])
    fig.set_title('')
    fig.set_ylabel(var)
    
    plt.subplot(1, 2, 2)
    fig = sns.distplot(tedtalk_df[var])
    fig.set_ylabel('views')
    fig.set_xlabel(var)

    plt.show()

In [ ]:
 tedtalk_df[['comments','duration']].describe().T

In [ ]:

#comments
#Finding IQR
Q3, Q1 = np.percentile(tedtalk_df['comments'], [75 ,25])
IQR = Q3 - Q1
upper = Q3 +1.5*IQR
lower = Q1 - 1.5*IQR

In [ ]:
tedtalk_df.loc[tedtalk_df['comments']>upper,'comments'] = upper

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.boxplot(tedtalk_df['comments'])
plt.subplot(1, 2, 2)
sns.distplot(tedtalk_df['comments'])

In [ ]:
tedtalk_df[['comments','duration']].describe().T

In [ ]:
#duration
#Finding IQR
Q3, Q1 = np.percentile(tedtalk_df['duration'], [75 ,25])
IQR = Q3 - Q1
upper = Q3 +1.5*IQR
lower = Q1 - 1.5*IQR

In [ ]:
tedtalk_df.loc[tedtalk_df['duration']>upper,'duration'] = upper
#borderizing the outliers

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.boxplot(tedtalk_df['duration'])
plt.subplot(1, 2, 2)
sns.distplot(tedtalk_df['duration'])
# plt.figure(figsize=(15,6))
# plt.subplot(1, 2, 1)
# sns.boxplot(tedtalk_df['duration'])
# plt.subplot(1, 2, 2)
# sns.distplot(tedtalk_df['duration'])

Here to treat outliers, we have used IQR method. Whatever the data were more than the Q3+1.5*IQR we tried equating it into upper range only. Only Comments and duration were treated this way.

To treat outliers in views cl=olumn which was a dependent column we tried using the Z score.

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.boxplot(tedtalk_df['views'])
plt.subplot(1, 2, 2)
sns.distplot(tedtalk_df['views'])


tedtalk_df = tedtalk_df[((tedtalk_df['views'] - tedtalk_df['views'].mean()) / tedtalk_df['views'].std()).abs() < 3]

plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.boxplot(tedtalk_df['views'])
plt.subplot(1, 2, 2)
sns.distplot(tedtalk_df['views'])

In [ ]:
ted_talk_2_df = tedtalk_df.copy()

In [ ]:
tedtalk_df = tedtalk_df.reset_index()

In [ ]:
tedtalk_df.info()

# **Feature Engineering**

In [ ]:
from datetime import date
today = pd.to_datetime(date.today(), format='%Y-%m-%d')

In [ ]:
#creating a column with video age
tedtalk_df['video_age'] = (today - tedtalk_df['published_date']).apply(lambda x: x.days)
# Creating a daily views column
tedtalk_df['views_per_day'] = tedtalk_df['views'] / ( tedtalk_df['video_age'] + 1 )

In [ ]:

#Finding average views according to first speaker
tedtalk_df['speaker_1_average_views'] = tedtalk_df['speaker_1'].map(list(tedtalk_df.groupby('speaker_1').agg({'views_per_day' : 'mean'}).
                                                         sort_values(['views_per_day'],ascending=False).to_dict().values())[0])

#Finding average views according to event 
tedtalk_df['event_average_views'] = tedtalk_df['event'].map(list(tedtalk_df.groupby('event').agg({'views_per_day' : 'mean'}).
                                                 sort_values(['views_per_day'],ascending=False).to_dict().values())[0])


In [ ]:
#Checking for unique topic
unique_topics=[]
for i in range(0,len(tedtalk_df)):
  temp=tedtalk_df['topics'][i]
  for i in temp:
    if(i not in unique_topics):
      unique_topics.append(i)

      
# Creating a dictionary with unique topics wrt average views
unique_topics_avg_view_dict={}
for topic in unique_topics:
  temp, count = 0, 0

  for i in range(0,len(tedtalk_df)):
    temp2=tedtalk_df['topics'][i]
    if(topic in temp2):
      temp+=tedtalk_df['views_per_day'][i]
      count+=1
  unique_topics_avg_view_dict[topic]=temp//count

# Creating a list and appending with average views wrt topic
topics_wise_average_views=[]
for i in range(0,len(tedtalk_df)):
  temp=0
  temp_topic=tedtalk_df['topics'][i]
  for element in temp_topic:
    temp+= unique_topics_avg_view_dict[element]
  
  topics_wise_average_views.append(temp//len(temp_topic))

In [ ]:

# Creating a new columns with average topic views
tedtalk_df['topics_wise_avg_views'] = pd.Series(topics_wise_average_views).values

# Creating a unique topics column which contain number of unique topics spoken on in an event
tedtalk_df['unique_topics'] = tedtalk_df['topics'].str.len()

In [ ]:
tedtalk_df['subtitles_count'] = tedtalk_df.available_lang.apply(lambda x: len(x))

tedtalk_df['Telecasted_after'] = (tedtalk_df['published_date'] - tedtalk_df['recorded_date']).apply(lambda x: x.days)
#creating a new variable 'telecasted after' which holds the difference between published date and recorded date


In [ ]:
tedtalk_df.head(2)

In [ ]:
tedtalk_df.info()

In [ ]:
df = tedtalk_df[['comments','duration','release_day','release_month','release_year',
                 'Telecasted_after','video_age','views_per_day','speaker_1_average_views','event_average_views','topics_wise_avg_views','unique_topics','subtitles_count']]
                 #taking only imp columns

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.columns

Now that morevariables are created in the above steps ,We can check for the ouliers once again to make sure that our data is free of outliers 

In [ ]:
def remove_outlier(df,column):
  
  plt.figure(figsize=(15,6))
  plt.subplot(1, 2, 1)
  plt.title('Before Treating outliers')
  sns.boxplot(df[column])
  plt.subplot(1, 2, 2)
  sns.distplot(df[column])
  df = df[((df[column] - df[column].mean()) / df[column].std()).abs() < 3]
  
  plt.figure(figsize=(15,6))
  
  plt.subplot(1, 2, 1)
  plt.title('After Treating outliers')
  sns.boxplot(df[column])
  plt.subplot(1, 2, 2)
  sns.distplot(df[column])

In [ ]:
df.describe().columns

In [ ]:
for column in df.describe().columns:
  remove_outlier(df,column)

In [ ]:
df.drop('Telecasted_after',axis=1,inplace=True)
#It has alot of outliers...better not to consider it

In [ ]:
df_2 = df.copy()

#**Checking the conditions for LR**

Now that Feature engineering is done, we need to start with train test split and start building our models. But since we are trying to start with linear regression in here, we need to check several criterias for LR.

In [ ]:
# defining in X and Y
y = df['views_per_day']
X = df.drop(columns=['views_per_day'])

In [ ]:
X.head(2)

In [ ]:
#to check multicollinearity, normal distribution, we need only numerical data, we dont want categorical variable
X_without_categorical_variable  = X[['comments', 'duration','video_age', 'speaker_1_average_views',
                                      'event_average_views', 'topics_wise_avg_views', 'unique_topics','subtitles_count']]

# to check linearity with the dependent variable.
df_without_categorical_variable =  df[['comments', 'duration','video_age','views_per_day','speaker_1_average_views',
                                      'event_average_views', 'topics_wise_avg_views', 'unique_topics','subtitles_count']]

**1) Checking linearity betweeen the dependent and independent variables**

In [ ]:
for column in df_without_categorical_variable:
  sns.lmplot(x=column, y="views_per_day", data=df_without_categorical_variable, order=1)
# plt.ylabel('Target')
# plt.xlabel('Independent variable')

**2) Multi collinearity should be avoided**

In [ ]:
DF = X_without_categorical_variable.copy()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
VIF_df = calc_vif(X[[i for i in DF.describe().columns ]])
VIF_df

In [ ]:
calc_vif(DF[[i for i in DF.describe().columns if i not in ['video_age','topics_wise_avg_views','subtitles_count'] ]])
#trying to leave which ever had >10

By VIF we have kept only few of the columns with less VIF score

**3)All independent variables shoould be normally distributed**

Here we checking from QQ plot 

#Applying Transformation

In [ ]:
#As we have more number of varibales,trying to create a function which does the plotting and transformation.
#But be carefull as we have several variables with positively skewed data and 'not-normal' data.
#Whenever we have positively skewed data we need to use the log transformation. Or else we can use Boxcox transformation.

#import libraries
import scipy.stats as stats
import pylab

def to_plot(column):
  plt.figure(figsize=(10,6))
  plt.subplot(1,2,1)
  sns.distplot(DF[column])
  plt.subplot(1,2,2)
  stats.probplot(DF[column],dist='norm',plot=pylab)
  plt.show()


def log_transform(column):
  print("Before Transformation")
  to_plot(column)
  # applying log transformation
  DF[column]=np.log1p(DF[column])
  #plotting
  print("After Transformation")
  to_plot(column)
  # stats.probplot()

def box_cox_transform(column):
  print("Before Transformation")
  to_plot(column)
  # applying boxcox transformation
  DF[column],parameters=stats.boxcox(DF[column])
  print("After Transformation")
  to_plot(column)

def square_root_transform(column):
  print("Before Transformation")
  to_plot(column)
  DF[column]=DF[column]**(1/2)
  print("After Transformation")
  to_plot(column)

def exponential_transform(column):
  print("Before Transformation")
  to_plot(column)
  DF[column]=DF[column]**(1/1.2)
  print("After Transformation")
  to_plot(column)


def power_transform(column):
  print("Applying Power Transformation.\n Before Transformation")
  to_plot(column)
  # applying power transformation
  DF[column]=np.power(DF[column],1/2)
  #plotting
  print("After Transformation")
  to_plot(column)
  # stats.probplot()


In [ ]:
for column in DF:
  to_plot(column)

In [ ]:
square_root_transform('comments')
box_cox_transform('duration')

square_root_transform('speaker_1_average_views')
box_cox_transform('event_average_views')

square_root_transform('unique_topics')

After applying the transformation and treating with VIF factor checking again for correlation

In [ ]:
DF.head()
#Which does not contain categorical and dependent variable

In [ ]:
corr = DF.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
DF.drop('event_average_views',axis=1,inplace=True)
#Strong correlation 
DF.drop('topics_wise_avg_views',axis=1,inplace=True)
DF.drop('subtitles_count',axis=1,inplace=True)

In [ ]:
corr = DF.corr()
corr.style.background_gradient(cmap='coolwarm')

We saw that event wise average views a=were highly correlated wih speaker average views, so we tried dropping that out.

In [ ]:
p = sns.pairplot(DF)

Just trying to visualize the DF treating with correlation and transformation.

**One Hot Encoding**

DF contains only the numeric columns, so we merge it with the categorical column. And we use One hot encoding to use the categorical columns in a better way.

In [ ]:
DF.columns

In [ ]:
features_df = DF.copy()

In [ ]:
features_df['release_day'] = df['release_day']
features_df['release_month'] = df['release_month']
features_df['release_year'] = df['release_year']

In [ ]:
features_df.head(2)

In [ ]:
features_df = pd.get_dummies(features_df, columns=['release_day', 'release_month', 'release_year'])

In [ ]:
features_df.head(2)

**Min Max Scaler**

Now everythin is done except normalization, we need to have the dat in same standards to get optimized results for the models. So we use Min max scaler

In [ ]:
#Min max scaler


column_names = list(features_df.columns)
# column_names

#taking columns to do the minmaxscaling
DF_scaled = pd.DataFrame()
#using standardization as both numeric columns are in different scale

scaler = MinMaxScaler()
scaled = scaler.fit_transform(features_df)
#print(scaled)
DF_scaled = pd.DataFrame(scaler.fit_transform(features_df))
DF_scaled.columns = column_names

In [ ]:

DF_scaled.head()

In [ ]:
DF.info()

**Dependent Variable**

All the normalization and transformations were done only for the independent variables. Now we try doing the same for dependent variable as well to get better result for our model

In [ ]:
df['views_per_day'].describe()

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
sns.distplot(df['views_per_day'])
plt.subplot(1,2,2)
stats.probplot(df['views_per_day'],dist='norm',plot=pylab)
plt.show()

In [ ]:

def plot_(col):
  plt.figure(figsize=(10,6))
  plt.subplot(1,2,1)
  sns.distplot(col)
  plt.subplot(1,2,2)
  stats.probplot(col,dist='norm',plot=pylab)
  plt.show()


In [ ]:
stats.probplot(df['views_per_day'],dist='norm',plot=pylab)
y1 = np.power(df['views_per_day'],1/3)
plot_(y1)
y2=np.power(df['views_per_day'],1/2)
plot_(y2)
y3=np.log1p(df['views_per_day'])
plot_(y3)
#y4=stats.boxcox(df['views_per_day'])
y5=df['views_per_day']**(1/2)
plot_(y5)
y6=df['views_per_day']**(1/1.2)
plot_(y6)

In [ ]:
# df['views_per_day'] = np.power(df['views_per_day'],1/3)
# plot_(df['views_per_day'])

Now our Data is all set to build model .Before that lets just do the train and test split

#Train and Test Split

We have already taken the dependent variable seperately, lets just equate it into y and X has ll the indeoendent variables.

In [ ]:
df.head(2)
y = y1
X = DF_scaled

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y,  test_size= 0.25, random_state= 0)


# Model Building 

In [ ]:
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR
import time
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import ensemble


In [ ]:
def Calculating_errors(y_train,y_pred_train,y_test,y_test_pred):
  MSE_test  = mean_squared_error(y_test, y_test_pred)
  print("MSE on test is" ,MSE_test)
  MSE_train  = mean_squared_error(y_train, y_pred_train)
  print("MSE on train is" ,MSE_train)
  RMSE_test = np.sqrt(MSE_test)
  print("RMSE on test is" ,RMSE_test)
  RMSE_train = np.sqrt(MSE_train)
  print("RMSE on train is" ,RMSE_train)
  print('Training MAE: {:0.2f}'.format(mean_absolute_error(y_train, y_pred_train)))
  print('Test MAE: {:0.2f}'.format(mean_absolute_error(y_test, y_test_pred)))


# Linear Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,lin_reg.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,lin_reg.predict(X_train)),4)}")

In [ ]:
y_pred_train_reg =lin_reg.predict(X_train)
y_test_pred_reg = lin_reg.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_pred_train_reg,y_test,y_test_pred_reg)

# Regularised linear regression

In [ ]:
!pip install catboost
!pip install lightgbm

In [ ]:
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.model_selection import train_test_split,GridSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,VotingRegressor,StackingRegressor


In [ ]:
elastic_param = {'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                 'l1_ratio' : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
                }

In [ ]:
elastic_grid = GridSearchCV(estimator=ElasticNet(),
                       param_grid = elastic_param,n_jobs=9,
                       cv = 5, verbose=2, scoring='r2')
elastic_grid.fit(X_train,y_train)

In [ ]:
optimal_elastic = elastic_grid.best_estimator_

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,optimal_elastic.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,optimal_elastic.predict(X_train)),4)}")

In [ ]:
y_pred_train_ela = optimal_elastic.predict(X_train)
y_test_pred_ela = optimal_elastic.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_pred_train_ela,y_test,y_test_pred_ela)

# Random Forest

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR

In [ ]:
rf = RandomForestRegressor(criterion='mae')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,rf.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,rf.predict(X_train)),4)}")

In [ ]:
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_rf,y_test,y_test_pred_rf)

# Optimal Random Forest

In [ ]:
rf_param = {
            'n_estimators':[100,250,500],
            'max_depth':[2,4,5,6],
            'max_leaf_nodes':[150]
           }

In [ ]:
rf_grid = GridSearchCV(estimator=RandomForestRegressor(),
                       param_grid = rf_param,n_jobs=9,
                       cv = 5, verbose=2, scoring='r2')
rf_grid.fit(X_train,y_train)

In [ ]:
optimal_rf = rf_grid.best_estimator_

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,optimal_rf.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,optimal_rf.predict(X_train)),4)}")

In [ ]:
y_train_pred_optimal_rf = optimal_rf.predict(X_train)
y_test_pred_optimal_rf = optimal_rf.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_optimal_rf,y_test,y_test_pred_optimal_rf)

# CatBoost Regression

In [ ]:
cat_reg = CatBoostRegressor()
cat_reg.fit(X_train,y_train)

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,cat_reg.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,cat_reg.predict(X_train)),4)}")

In [ ]:
y_train_pred_cat = cat_reg.predict(X_train)
y_test_pred_cat = cat_reg.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_cat,y_test,y_test_pred_cat)

# Optimal catboost regression

In [ ]:
cat_param = {'depth': [2,4,5],
             'learning_rate' : [0.01,0.05,0.1],
             'iterations'    : [150,200]
            }

In [ ]:
ct_grid = GridSearchCV(estimator=cat_reg,
                       param_grid = cat_param,n_jobs=9,
                       cv = 5, verbose=2, scoring='r2')

In [ ]:
ct_grid.fit(X_train,y_train)

In [ ]:
optimal_cat = ct_grid.best_estimator_

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,optimal_cat.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,optimal_cat.predict(X_train)),4)}")

In [ ]:
y_train_pred_optcat = optimal_cat.predict(X_train)
y_test_pred_optcat = optimal_cat.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_optcat,y_test,y_test_pred_optcat)

# XG Boost Regression

In [ ]:
xgb_reg = XGBRegressor()

In [ ]:
xgb_reg.fit(X_train,y_train)

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,xgb_reg.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,xgb_reg.predict(X_train)),4)}")

In [ ]:
y_train_pred_xgb = xgb_reg.predict(X_train)
y_test_pred_xgb = xgb_reg.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_xgb,y_test,y_test_pred_xgb)

# Optimised XGboost 

In [ ]:
xgb_param = {'n_estimators': [50,100,150,200],
            'max_depth': [2,4,5,10],
            'learning_rate':[0.01,0.05,0.1]
            }

In [ ]:
xgb_grid = GridSearchCV(estimator=xgb_reg,
                       param_grid = xgb_param,n_jobs=9,
                       cv = 5, verbose=2, scoring='r2')

In [ ]:
xgb_grid.fit(X_train,y_train)

In [ ]:
optimal_xgb = xgb_grid.best_estimator_

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,optimal_xgb.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,optimal_xgb.predict(X_train)),4)}")

In [ ]:
y_train_pred_optxgb = optimal_xgb.predict(X_train)
y_test_pred_optxgb = optimal_xgb.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_optxgb,y_test,y_test_pred_optxgb)

# LGBM

In [ ]:
lgbm_reg = LGBMRegressor()
lgbm_reg.fit(X_train,y_train)

In [ ]:
r2_score(y_train,lgbm_reg.predict(X_train))

In [ ]:
r2_score(y_test,lgbm_reg.predict(X_test))

In [ ]:
y_train_pred_lgbm = lgbm_reg.predict(X_train)
y_test_pred_lgbm = lgbm_reg.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_lgbm,y_test,y_test_pred_lgbm)

# Optimal LGBM

In [ ]:
lgbm_param = {
              "n_estimators" :[50,100,150],
              'num_leaves': [6,8,12,16],
              "max_depth": [2,4,5],
              "learning_rate": [0.01,0.05,0.1]
              }

In [ ]:
lgbm_grid = GridSearchCV(estimator=lgbm_reg,
                       param_grid = lgbm_param,n_jobs=6,
                       cv = 5, verbose=2, scoring='r2')

In [ ]:
lgbm_grid.fit(X_train,y_train)

In [ ]:
lgbm_optimal = lgbm_grid.best_estimator_

In [ ]:
r2_score(y_test,lgbm_optimal.predict(X_test))

In [ ]:
r2_score(y_train,lgbm_optimal.predict(X_train))

In [ ]:
y_train_pred_lgbm_optimal = lgbm_optimal.predict(X_train)
y_test_pred_lgbm_optimal = lgbm_optimal.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_lgbm_optimal,y_test,y_test_pred_lgbm_optimal)

# Extra Tree Regression

In [ ]:
extra_tree_reg = ExtraTreesRegressor(criterion='mae', max_depth=30, n_estimators=200, min_samples_leaf=2, min_samples_split=6)
extra_tree_reg.fit(X_train, y_train)

In [ ]:
print(f"The r2 score for testing is {round(r2_score(y_test,extra_tree_reg.predict(X_test)),4)}")
print(f"The r2 score for training is {round(r2_score(y_train,extra_tree_reg.predict(X_train)),4)}")

In [ ]:
y_train_pred_extra = extra_tree_reg.predict(X_train)
y_test_pred_extra = extra_tree_reg.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_extra,y_test,y_test_pred_cat)

# Optimal Extra Tree Regression

In [ ]:
#Extra Trees Regressor paramdict
# Number of trees
n_estimators = [50,80,100]
# Maximum depth of trees
max_depth = [4,6,8]
# Minimum number of samples required to split a node
min_samples_split = [50,100,150]
# Minimum number of samples required at each leaf node
min_samples_leaf = [40,50]
# HYperparameter Dict
param_dict = {'n_estimators' : n_estimators,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf}

In [ ]:
# Create an instance of the Extratrees regressor
et_model = ExtraTreesRegressor(criterion='mae')

#RandomSearch
et_random = RandomizedSearchCV(et_model,param_dict,verbose=2,cv=5)
et_random.fit(X_train,y_train)

In [ ]:
et_optimal_model = et_random.best_estimator_

In [ ]:
y_train_pred_etopt = et_optimal_model.predict(X_train)
y_test_pred_etopt = et_optimal_model.predict(X_test)

In [ ]:
Calculating_errors(y_train,y_train_pred_etopt,y_test,y_test_pred_etopt)

In [ ]:
# make predictions for test data
y_hat = et_optimal_model.predict(X_test)
print(f'r_sqr value for train: {et_optimal_model.score(X_train, y_train)}')
r_squared= r2_score(y_test,y_hat)
#Calculate Adjusted R-sqaured
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
print(f'For test the R_Squared for ExtraTreesRegressor is {r_squared} and adjusted R_Squared is {adjusted_r_squared}')

**Comparing all models with model performance**

In [ ]:
regressors = [optimal_elastic,optimal_rf,optimal_cat,xgb_reg,lgbm_optimal,et_optimal_model]
reg_names = ['Regularized Linear regression','RandomForest','CatBoost','XGBoost','LGBM','Extra tree regressor']
training,testing = [],[]
for i in regressors:
    tr = round(r2_score(y_train,i.predict(X_train)),3)
    ts = round(r2_score(y_test,i.predict(X_test)),3)
    training.append(tr)
    testing.append(ts)

In [ ]:
diff = np.array(training)-np.array(testing)

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(range(0,len(regressors)),training,'--o',lw=2,label='Training')
plt.plot(range(0,len(regressors)),testing,'-o',lw=2,label='Testing')
plt.xticks(range(0,len(regressors)), reg_names, rotation=45,fontsize=14)
plt.axvline(np.argmin(diff),linestyle=':', color='black', label=f'Best performing model')
plt.ylabel("Scores")
plt.title("Comparing training and testing r2-scores for our models")
plt.grid(True)
plt.legend(loc='best',fontsize=12);

We can observe that our modt models are overfitting, they performed alot better in traing but their performance is poor in testing. But comparitvely Regularized linear regression is performing well in traing and testing.

**Evaluation metric for all models (in comaprision)**

In [ ]:
#Defining all the models
models = [
           ['Regularized Linear Regression', elastic_grid.best_estimator_],
           ['Optimal Random Forest ', rf_grid.best_estimator_],
           ['LGBM ', lgbm_grid.best_estimator_],
           ['Catboost ',  ct_grid.best_estimator_],
           ['XGBoost ',xgb_reg],    
           ['Extra tree regressor ', et_random.best_estimator_]          
        ]

In [ ]:
#Let's run all the models and store the scores
model_1_data = []
for name,model in models :
    model_data = {}
    model.random_state = 42
    model_data["Name"] = name
    model.fit(X_train,y_train)
    model_data["MAE_train"] = round(metrics.mean_absolute_error(y_train, model.predict(X_train)),4)
    model_data["MAE_test"] = round(metrics.mean_absolute_error(y_test, model.predict(X_test)),4)
    model_data["MSE_train"] = round(metrics.mean_squared_error(y_train, model.predict(X_train)),4)
    model_data["MSE_test"] = round(metrics.mean_squared_error(y_test, model.predict(X_test)),4)
    model_data["R2_Score_train"] = round(r2_score(y_train,model.predict(X_train)),4)
    model_data["R2_Score_test"] = round(r2_score(y_test,model.predict(X_test)),4)
    model_data['Adjusted_R2_score_train'] = 1 - (1-(r2_score(y_train,model.predict(X_train))))*(len(y)-1)/(len(y)-X.shape[1]-1)
    model_data['Adjusted_R2_score_test'] = 1 - (1-(r2_score(y_test,model.predict(X_test))))*(len(y)-1)/(len(y)-X.shape[1]-1)
    model_data["RMSE_Score_train"] = round(np.sqrt(mean_squared_error(y_train,model.predict(X_train))),4)
    model_data["RMSE_Score_test"] = round(np.sqrt(mean_squared_error(y_test,model.predict(X_test))),4)
    model_1_data.append(model_data)

In [ ]:
results = pd.DataFrame(model_1_data)
results 

# Conclusion 

That's it! We reached the end of our exercise.

Started with loading the data so far we have done EDA ,feature engineering , data cleaning, target encoding and one hot encoding of categorical columns, feature selection and then model building.

**So far we have modelled on**

Lasso Regressor

Ridge Regressor


Random Forest Regressor

Extra Tree Regressor

Gradient Boosting Regressor

XGB Regressor

In all the features speaker_wise_avg_views is most important this implies that speakers are directly impacting the views.


After comparing all the models we can conclude that Optimal Random Forest and aswell as Extra Tree is the best performer.